# RAG

# Installing & Importing Libraries

In [1]:
# insalling libraries
!pip install -q -U \
  langchain \
  langchain-core \
  langchain-community \
  langchain-openai \
  langchain-text-splitters \
  chromadb \
  gradio \
  python-dotenv \
  tiktoken \
  openai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("لم يتم العثور على OPENAI_API_KEY. تأكد أنك وضعته في .env أو في متغيرات البيئة.")

# Loading & Splitting Data (Chunking)

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

/workspaces/rag-test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
DATA_FILE_PATH = "lawazem-info.txt"   # عدّله إذا اسم الملف مختلف
PERSIST_DIR = "chroma_db"            # لتخزين قاعدة Chroma على القرص (اختياري لكنه مفيد)

# 1) Load
loader = TextLoader(DATA_FILE_PATH, encoding="utf-8")
raw_docs = loader.load()

# 2) Split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=80,
    separators=["\n\n", "\n", "—", "-", "•", ".", "؟", "!", "،", " "],
)
docs = splitter.split_documents(raw_docs)

print("عدد المقاطع:", len(docs))
print("مثال مقطع:\n", docs[0].page_content[:300], "...")

عدد المقاطع: 4
مثال مقطع:
 🏪 معرض لوازم للأدوات المنزلية والأجهزة الكهربائية

🔹 النشأة:
تأسست شركة لوازم في عام 2023 في مدينة أسيوط، مصر. بدأت بفرع صغير في وسط البلد، ومع مرور الوقت توسعت لتصبح واحدة من أبرز معارض الأدوات المنزلية في صعيد مصر.

🔹 المقر الرئيسي:
العنوان: ش سيتي، أسيوط، مصر.
الهاتف: +201033103539
البريد الإلكتر ...


In [7]:
embeddings = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-small"
)

# إنشاء/تحميل قاعدة Chroma
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=PERSIST_DIR
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("✅ تم تجهيز الـ retriever")

✅ تم تجهيز الـ retriever


# Testing the Retrival

In [8]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",   # غيّرها إذا تريد نموذج آخر
    temperature=0.2
)

prompt = ChatPromptTemplate.from_template(
    """أنت مساعد مبيعات لمتجر.
أجب بالعربية وباختصار ووضوح اعتمادًا فقط على السياق التالي.
إذا لم تجد الإجابة في السياق، قل: "لا أملك معلومة كافية من البيانات الحالية."

السياق:
{context}

السؤال:
{question}

الإجابة:"""
)

def format_docs(docs):
    # تنسيق المستندات داخل السياق
    return "\n\n".join([f"- {d.page_content.strip()}" for d in docs])

# هذا الـ chain يأخذ سؤال (string) ويرجع جواب (string)
rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Building & Testing RAG using LangChain

In [9]:
def ask_with_sources(question: str):
    # نجلب المستندات يدويًا حتى نعرض المصادر
    if hasattr(retriever, "invoke"):
        source_docs = retriever.invoke(question)
    else:
        source_docs = retriever.get_relevant_documents(question)

    answer = rag_chain.invoke(question)

    # استخراج المصادر (إن وجدت)
    sources = []
    for d in source_docs:
        src = d.metadata.get("source")
        if src and src not in sources:
            sources.append(src)

    # سنيبتس قصيرة للمراجعة
    snippets = []
    for i, d in enumerate(source_docs[:4], start=1):
        snippets.append(f"[{i}] {d.page_content[:220].strip()}...")

    return answer, ("\n".join(sources) if sources else "لا توجد مصادر (metadata) واضحة."), "\n\n".join(snippets)


# اختبار سريع
ans, srcs, snips = ask_with_sources("عندكم كاتل كهرباء؟")
print("الإجابة:\n", ans)
print("\nالمصادر:\n", srcs)
print("\nمقاطع مستخدمة:\n", snips)

الإجابة:
 نعم، لدينا كاتل كهربائي سعة 1.7 لتر بسعر 850 جنيه مصري.

المصادر:
 lawazem-info.txt

مقاطع مستخدمة:
 [1] 🔹 المنتجات:
- أطقم عشاء بورسلين (18 قطعة): سعرها يبدأ من 1500 جنيه مصري.
- أواني طهي ستانلس ستيل (طقم 10 قطع): السعر حوالي 2200 جنيه مصري.
- كاتل كهربائي 1.7 لتر: سعره 850 جنيه مصري.
- خلاط كهربائي 600 وات مع مطحنة: السع...

[2] - طاسة تيفال 28 سم: السعر 750 جنيه مصري.
- سخان مياه كهربائي 50 لتر: السعر 3700 جنيه مصري....

[3] 🔹 سياسة الاستبدال والإرجاع:
- يمكن استبدال المنتجات أو إرجاعها خلال 14 يوم من تاريخ الشراء مع الاحتفاظ بالفاتورة.
- الضمان يشمل الأجهزة الكهربائية من سنة إلى ثلاث سنوات حسب نوع المنتج.

🔹 ساعات العمل:
- السبت – الخميس: 1...

[4] 🏪 معرض لوازم للأدوات المنزلية والأجهزة الكهربائية

🔹 النشأة:
تأسست شركة لوازم في عام 2023 في مدينة أسيوط، مصر. بدأت بفرع صغير في وسط البلد، ومع مرور الوقت توسعت لتصبح واحدة من أبرز معارض الأدوات المنزلية في صعيد مصر.

🔹...


In [25]:
chain_test_query = "عندكم كاتل كهرباء؟"
try:
    result = qa_chain.invoke({"question": chain_test_query})

    # Print the answer and sources from the result dictionary
    print("\n--- Answer ---")
    print(result.get("answer", "No answer generated."))

    print("\n--- Sources ---")
    print(result.get("sources", "No sources identified."))

    # Optionally print snippets from the source documents returned
    if "source_documents" in result:
        print("\n--- Source Document Snippets ---")
        for i, doc in enumerate(result["source_documents"]):
            content_snippet = doc.page_content[:250].strip()
            print(f"Doc {i+1}: {content_snippet}")

except Exception as e:
    print(f"\nAn error occurred while running the chain: {e}")



> Entering new RetrievalQAWithSourcesChain chain...

An error occurred while running the chain: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}


# Building Gradio UI

In [11]:
import gradio as gr

def gradio_chat(user_question):
    if not user_question.strip():
        return "اكتب سؤالك أولاً.", "", ""

    answer, sources, snippets = ask_with_sources(user_question)
    return answer, sources, snippets

ui = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(label="سؤالك", placeholder="مثال: هل عندكم كاتل كهرباء؟"),
    outputs=[
        gr.Textbox(label="الإجابة"),
        gr.Textbox(label="المصادر"),
        gr.Textbox(label="مقاطع من البيانات (للتأكد)")
    ],
    title="RAG + Chroma + OpenAI (متوافق مع LangChain 1.x بدون chains)",
    description="يسحب الإجابة من ملف النص فقط ويعرض مصادر/مقاطع داعمة."
)

ui.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e8b24a5de5a6f077f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
